In [1]:
import numpy as np
import pandas as pd
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [2]:
advertise_data = pd.read_csv("advertising.csv")
advertise_data

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [3]:
x = advertise_data.drop("sales", axis=1)
y = advertise_data["sales"]

In [4]:
from sklearn.preprocessing import PolynomialFeatures
poly_feat = PolynomialFeatures(degree=3, include_bias=False)
poly_feat = poly_feat.fit_transform(x)

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(poly_feat, y, test_size=0.2, random_state=34)

# Scaling the data with the help of Standard Scaler

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(x_train)
x_train = scaler.transform(x_train)

In [7]:
x_test = scaler.transform(x_test)

# Implementing the Ridge Regularization (L2)

In [8]:
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=10)
ridge_model.fit(x_train, y_train)

Ridge(alpha=10)

In [9]:
test_pred = ridge_model.predict(x_test)

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_test, test_pred)
mae = mean_absolute_error(y_test, test_pred)
rmse = np.sqrt(mse)

In [11]:
rmse, mae

(0.535329170912303, 0.42912504481797453)

# Implementing the Ridge Cross-Validation Regularization (L2)

In [12]:
from sklearn.linear_model import RidgeCV

ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10), scoring='neg_mean_absolute_error')
ridge_cv_model.fit(x_train, y_train)

RidgeCV(alphas=(0.1, 1.0, 10), scoring='neg_mean_absolute_error')

In [13]:
ridge_cv_model.alpha_

0.1

In [14]:
from sklearn import metrics
metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [15]:
test_pred = ridge_cv_model.predict(x_test)

In [16]:
mse = mean_squared_error(y_test, test_pred)
mae = mean_absolute_error(y_test, test_pred)
rmse = np.sqrt(mse)

In [17]:
rmse, mae

(0.3706965768837069, 0.3095061865502733)

In [18]:
ridge_cv_model.coef_

array([ 6.1203085 ,  0.60832034,  0.33619906, -7.28229895,  4.20105384,
       -0.80195842, -0.51697578,  0.09891712, -0.12666945,  2.99933083,
       -0.81112408,  0.43465747,  0.37038799, -0.25364973,  0.26530192,
        0.16865861,  0.01082211,  0.08567836, -0.07766302])

# Implementing the Lasso Cross-Validation Regularization (L1)

In [19]:
from sklearn.linear_model import LassoCV

lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, cv=10, max_iter=2000)
lasso_cv_model.fit(x_train, y_train)

LassoCV(cv=10, max_iter=2000)

In [20]:
lasso_cv_model.alpha_

0.00493460454354785

In [21]:
lasso_cv_model.coef_

array([ 5.66532102e+00,  4.31693717e-01,  1.57167533e-01, -6.08618741e+00,
        3.77415636e+00, -2.37219577e-01,  0.00000000e+00,  3.97527677e-03,
        6.26340115e-03,  2.14761704e+00, -1.67798595e-01, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  3.77994172e-02,  3.39944003e-02,
        0.00000000e+00,  0.00000000e+00,  1.66430246e-03])

In [22]:
test_pred = lasso_cv_model.predict(x_test)

In [23]:
mse = mean_squared_error(y_test, test_pred)
mae = mean_absolute_error(y_test, test_pred)
rmse = np.sqrt(mse)

In [24]:
rmse, mae

(0.40159131543909066, 0.3358675193166209)

# Implementing the Elastic Net Cross-Validation Regularization (L1+L2)

In [25]:
from sklearn.linear_model import ElasticNetCV
elastic_cv_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], eps=0.001, n_alphas=100, max_iter=2000)
elastic_cv_model.fit(x_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=2000)

In [26]:
elastic_cv_model.l1_ratio_

1.0

In [27]:
elastic_cv_model.alpha_

0.00493460454354785

In [28]:
elastic_cv_model.coef_

array([ 5.66532102e+00,  4.31693717e-01,  1.57167533e-01, -6.08618741e+00,
        3.77415636e+00, -2.37219577e-01,  0.00000000e+00,  3.97527677e-03,
        6.26340115e-03,  2.14761704e+00, -1.67798595e-01, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  3.77994172e-02,  3.39944003e-02,
        0.00000000e+00,  0.00000000e+00,  1.66430246e-03])

In [29]:
test_pred = elastic_cv_model.predict(x_test)

In [30]:
mse = mean_squared_error(y_test, test_pred)
mae = mean_absolute_error(y_test, test_pred)
rmse = np.sqrt(mse)
rmse, mae

(0.40159131543909066, 0.3358675193166209)